# 🗺️ Folium 지도 시각화 심화 실습

이번 시간에는 **Folium**을 활용해 지도 시각화를 실습합니다.

Folium은 **Python에서 지도를 시각화할 수 있는 라이브러리**로, HTML 기반의 인터랙티브(Interactive) 지도를 만들어 줍니다.  

> 🎯 학습 목표  
> - `folium.Map()`으로 **기본 지도 생성하기**  
> - `Marker`, `Icon`, `DivIcon`으로 **다양한 마커 표시**  
> - `zoom_start`, `location` 등 **지도 중심 설정 이해하기**  
> - 여러 지점을 **DataFrame으로 자동 반복 시각화**  

💡 **중요 개념 정리**
| 매개변수 | 설명 |
|-----------|------|
| `location` | [위도, 경도] 형식의 리스트로 지도 중심을 설정 |
| `zoom_start` | 지도 확대 수준 (숫자가 클수록 더 가까이 확대됨) |
| `icon` | 마커의 색상 및 모양 지정 |
| `popup` | 마커 클릭 시 표시되는 텍스트 |
| `DivIcon` | 지도 위에 **텍스트 라벨** 표시할 때 사용 |

---

## 1. 기본 지도 생성

`folium.Map()`을 사용하여 지도 객체를 만들고 `location`과 `zoom_start`를 지정합니다.


In [ ]:
# 필수 패키지 설치
!pip install folium geokakao pandas --quiet

# 라이브러리 불러오기
import folium
import geokakao as gk
import pandas as pd

In [ ]:
# 1) 지도 중심 좌표 설정
center = [37.57, 126.98]  # 서울 종로구 근처
print("지도 중심 좌표:", center)

지도 중심 좌표: [37.57, 126.98]


In [ ]:
# 2) 지도 객체 생성
# - location: 중심 좌표
# - zoom_start: 확대 레벨 (1=전세계, 13=도시 수준, 16=건물 수준)
map = folium.Map(location=center, zoom_start=13)

In [ ]:
# 3) 출력 확인
print("folium.Map 객체 생성 완료! 아래에 지도가 표시됩니다 👇")
map

folium.Map 객체 생성 완료! 아래에 지도가 표시됩니다 👇


## 2. 주소를 중심으로 지도 표시하기

이번에는 **지명(주소)** 을 입력하고 이를 **위도/경도 좌표로 변환**하여 지도 중심으로 설정합니다.

> 🧩 사용 함수  
> - `gk.convert_address_to_coordinates(주소)` → `[위도, 경도]`

In [ ]:
# 주소 → 좌표로 변환
address = "서울 종로구 사직로 161"  # 경복궁
loc = gk.convert_address_to_coordinates(address)
print(f"주소 '{address}'의 변환 좌표:", loc)

주소 '서울 종로구 사직로 161'의 변환 좌표: ('37.5759040910202', '126.976842133821')


In [ ]:
# 지도 생성
map = folium.Map(location=loc, zoom_start=16)
print("지도 중심(위도, 경도):", map.location)
print("줌 레벨(zoom_start):", map.options['zoom'])

지도 중심(위도, 경도): [37.5759040910202, 126.976842133821]
줌 레벨(zoom_start): 16


In [ ]:
# 마커(Marker) 추가
# - location: 표시할 좌표
# - popup: 클릭 시 나올 정보
folium.Marker(location=loc, popup="경복궁").add_to(map)
print("마커 추가 완료 — popup='경복궁'")

map

마커 추가 완료 — popup='경복궁'


## 3. 마커 스타일 변경 및 텍스트 추가

Folium은 단순 마커 외에도, `Icon` 옵션으로 색상/모양을 바꾸거나 `DivIcon`으로 텍스트를 표시할 수 있습니다.

In [ ]:
# 강릉 경포해수욕장 좌표 구하기
address = "강원특별자치도 강릉시 창해로 514"
loc = gk.convert_address_to_coordinates(address)
print(f"'{address}' 의 좌표:", loc)

'강원특별자치도 강릉시 창해로 514' 의 좌표: ('37.8057701120176', '128.907481194757')


In [ ]:
# 지도 생성
map = folium.Map(location=loc, zoom_start=13)
print(f"지도 중심 좌표: {map.location}, 확대레벨: {map.options['zoom']}")

지도 중심 좌표: [37.8057701120176, 128.907481194757], 확대레벨: 13


In [ ]:
# (1) 별모양 붉은 마커 추가
folium.Marker(
    location=loc,
    icon=folium.Icon(color="red", icon="star")
).add_to(map)
print("붉은 별 마커 추가 완료!")

붉은 별 마커 추가 완료!


In [ ]:
# (2) DivIcon으로 텍스트 추가
html_label = '<div style="font-size:12px; color:blue; background-color:rgba(255,255,255,0.6);"><b>경포해수욕장</b></div>'
folium.Marker(location=loc, icon=folium.DivIcon(html=html_label)).add_to(map)
print("텍스트 라벨 추가 완료!")

map

텍스트 라벨 추가 완료!


## 4. DataFrame을 이용한 여러 관광지 표시

이번에는 **관광지 리스트를 DataFrame 형태로 저장**하고, `for` 반복문으로 여러 지점을 한 번에 지도 위에 표시해 보겠습니다.

In [ ]:
# 제주 관광지 예시 데이터
names = ['용두암', '성산일출봉', '정방폭포', '중문관광단지', '한라산1100고지', '차귀도']
addr = [
    '제주시 용두암길 15',
    '서귀포시 성산읍 성산리',
    '서귀포시 동홍동 299-3',
    '서귀포시 중문동 2624-1',
    '서귀포시 색달동 산1-2',
    '제주시 한경면 고산리 산 117'
]

In [ ]:
# DataFrame 생성
df = pd.DataFrame({'관광지명': names, '주소': addr})
print("초기 DataFrame 미리보기:")
print(df.head(), "\n")

초기 DataFrame 미리보기:
        관광지명               주소
0        용두암      제주시 용두암길 15
1      성산일출봉     서귀포시 성산읍 성산리
2       정방폭포   서귀포시 동홍동 299-3
3     중문관광단지  서귀포시 중문동 2624-1
4  한라산1100고지    서귀포시 색달동 산1-2 



In [ ]:
# 주소를 위도/경도로 변환하여 열 추가
gk.add_coordinates_to_dataframe(df, '주소')

In [ ]:
# 좌표 데이터 확인
print("좌표 추가 후 DataFrame:")
print(df.head(), "\n")

좌표 추가 후 DataFrame:
        관광지명               주소   decimalLatitude  decimalLongitude
0        용두암      제주시 용두암길 15  33.5148035919525  126.511798046978
1      성산일출봉     서귀포시 성산읍 성산리  33.4641549468628  126.933094402743
2       정방폭포   서귀포시 동홍동 299-3  33.2450439717872  126.571611812583
3     중문관광단지  서귀포시 중문동 2624-1  33.2458014088806  126.420647253077
4  한라산1100고지    서귀포시 색달동 산1-2  33.3575245371214  126.462896523457 



In [ ]:
# 좌푯값을 숫자로 변환
df['decimalLatitude'] = pd.to_numeric(df['decimalLatitude'])
df['decimalLongitude'] = pd.to_numeric(df['decimalLongitude'])
print("좌표 데이터 타입 확인:")
print(df.dtypes[['decimalLatitude', 'decimalLongitude']], "\n")

좌표 데이터 타입 확인:
decimalLatitude     float64
decimalLongitude    float64
dtype: object 



In [ ]:
# 지도 중심 계산
center = [df['decimalLatitude'].mean(), df['decimalLongitude'].mean()]
print(f"지도 중심 좌표 (평균값): {center}")

지도 중심 좌표 (평균값): [np.float64(33.35646904072083), np.float64(126.50771527972067)]


In [ ]:
# 지도 생성
map = folium.Map(location=center, zoom_start=10)

In [ ]:
# 각 관광지마다 마커 & 텍스트 표시
print("\n 지도에 마커 추가 중...")
for i in range(len(df)):
    loc = [df.loc[i, 'decimalLatitude'], df.loc[i, 'decimalLongitude']]
    folium.Marker(location=loc, icon=folium.Icon(color='red', icon='star')).add_to(map)
    html_label = f'<div style="font-size:12px; color:blue;"><b>{df.loc[i, "관광지명"]}</b></div>'
    folium.Marker(location=loc, icon=folium.DivIcon(html=html_label)).add_to(map)
    print(f"  - {df.loc[i, '관광지명']} 마커 추가 완료!")

print("\n 모든 관광지 마커 추가 완료!")
map


 지도에 마커 추가 중...
  - 용두암 마커 추가 완료!
  - 성산일출봉 마커 추가 완료!
  - 정방폭포 마커 추가 완료!
  - 중문관광단지 마커 추가 완료!
  - 한라산1100고지 마커 추가 완료!
  - 차귀도 마커 추가 완료!

 모든 관광지 마커 추가 완료!


In [ ]:
# 관광지 데이터 확인용 요약
print("총 관광지 개수:", len(df))
print("\n관광지명 리스트:", list(df['관광지명']))
print("\n위도 평균:", round(df['decimalLatitude'].mean(), 5))
print("경도 평균:", round(df['decimalLongitude'].mean(), 5))

총 관광지 개수: 6

관광지명 리스트: ['용두암', '성산일출봉', '정방폭포', '중문관광단지', '한라산1100고지', '차귀도']

위도 평균: 33.35647
경도 평균: 126.50772


# 💡 Folium 지도 시각화 응용 가이드 (심화 이론 정리)

이 섹션에서는 Folium을 활용할 때 알아두면 좋은 **고급 설정 옵션과 시각화 확장 아이디어**를 정리합니다.  
학생들은 이를 바탕으로 지도를 자신만의 스타일로 꾸미거나, 데이터 기반 시각화를 심화시킬 수 있습니다.

---

## 1️⃣ 지도(Map) 객체 확장 옵션

Folium의 `folium.Map()`은 단순한 지도 객체를 생성하는 것 외에도  
다양한 속성을 통해 지도의 시각적 스타일과 조작성을 개선할 수 있습니다.

| 옵션명 | 설명 |
|--------|------|
| `location` | 지도 중심 좌표로 `[위도, 경도]` 형태로 지정합니다. |
| `zoom_start` | 초기 확대 수준 (1 = 전 세계, 18 = 건물 수준). |
| `tiles` | 지도 배경 스타일을 선택합니다.<br>예: `"OpenStreetMap"`, `"Stamen Terrain"`, `"CartoDB positron"`, `"Stamen Toner"`, `"CartoDB dark_matter"` |
| `control_scale` | 지도의 하단에 **거리 스케일바**를 표시할지 여부를 결정합니다. |
| `min_zoom / max_zoom` | 사용자가 확대·축소 가능한 최소/최대 범위를 제한합니다. |
| `attr` | 커스텀 지도 타일을 사용할 때 저작권 표시를 추가할 수 있습니다. |

> 💡 `tiles` 속성은 시각적인 느낌을 완전히 바꿀 수 있으므로,  
> 프로젝트 목적(관광, 기상, 통계 등)에 맞는 스타일을 선택하는 것이 중요합니다.

---

In [ ]:
# 🌍 Folium 기본 지도 설정 예제
import folium

# 1️⃣ 지도 중심 좌표 지정 — 서울 종로구 근처
center = [37.57, 126.98]
print("지도 중심 좌표:", center)

# 2️⃣ 다양한 옵션과 스타일 적용
map1 = folium.Map(
    location=center,           # 중심 좌표
    zoom_start=13,             # 확대 수준 (1~18)
    tiles="CartoDB positron",    # 지도 스타일
    control_scale=True,        # 하단 스케일바 표시
    min_zoom=8,                # 최소 확대 수준 제한
    max_zoom=17                # 최대 확대 수준 제한
)
print("Folium 지도 객체 생성 완료!")

# 3️⃣ 지도 출력
map1

지도 중심 좌표: [37.57, 126.98]
Folium 지도 객체 생성 완료!


---

## 2️⃣ 마커(Marker) 관련 속성 및 커스터마이징

Folium의 `Marker()`는 지도 위 특정 지점을 표시하는 가장 기본적인 객체입니다.  
이 마커에 다양한 속성을 지정하여 색상, 모양, 아이콘을 자유롭게 변경할 수 있습니다.

| 속성명 | 설명 |
|--------|------|
| `location` | 마커가 표시될 좌표 `[위도, 경도]` |
| `popup` | 마커 클릭 시 표시되는 텍스트 또는 HTML |
| `tooltip` | 마우스 커서를 올렸을 때 잠깐 표시되는 설명 텍스트 |
| `icon` | `folium.Icon()` 객체로 마커의 색상·모양을 지정 |
| `draggable` | `True`로 설정 시 사용자가 마커를 드래그할 수 있음 |

### 🧩 `Icon()` 객체 주요 속성
| 속성명 | 설명 |
|--------|------|
| `color` | 마커 색상 지정 (`'red'`, `'blue'`, `'green'`, `'purple'` 등) |
| `icon` | 아이콘 형태 (`'star'`, `'flag'`, `'home'`, `'camera'` 등) |
| `prefix` | `'fa'`를 지정하면 Font Awesome 아이콘 세트를 사용 가능 |
| `icon_color` | 아이콘 내부의 색상 지정 (기본값은 흰색) |

> 💡 Folium의 기본 아이콘 외에도 Font Awesome 아이콘을 자유롭게 사용할 수 있습니다.

---

In [ ]:
# 📍 Folium Marker 응용 예제
import folium

# 서울시청 좌표
loc = [37.5665, 126.9780]

# 지도 생성
map2 = folium.Map(location=loc, zoom_start=15)
print("지도 중심:", map2.location)

# 1️⃣ 기본 마커
folium.Marker(
    location=loc,
    popup="서울시청 (기본 마커)"
).add_to(map2)

# 2️⃣ 색상 + 아이콘 커스터마이징
folium.Marker(
    location=[37.5668, 126.9785],
    popup="보라색 집 아이콘",
    icon=folium.Icon(color="purple", icon="home", prefix="fa")
).add_to(map2)

# 3️⃣ 드래그 가능한 마커
folium.Marker(
    location=[37.5671, 126.9790],
    popup="이 마커는 드래그 가능!",
    draggable=True,
    icon=folium.Icon(color="green", icon="flag")
).add_to(map2)

print("총 3개의 마커가 추가되었습니다!")
map2

지도 중심: [37.5665, 126.978]
총 3개의 마커가 추가되었습니다!


---

## 3️⃣ 텍스트 라벨(DivIcon) 활용

`DivIcon`은 지도 위에 **글자, HTML, 이모티콘, 스타일이 적용된 요소**를 직접 표시할 수 있는 강력한 기능입니다.  
HTML을 이용해 글자 크기, 색상, 배경 등을 자유롭게 꾸밀 수 있습니다.

| 속성명 | 설명 |
|--------|------|
| `html` | 실제로 표시할 HTML 코드 문자열 |
| `icon_anchor` | 텍스트 위치 기준점을 지정 (`(x, y)` 좌표) |
| `class_name` | CSS 클래스를 지정할 수 있음 (기본값: `"empty"`) |

### ✨ 주요 스타일 속성 (HTML 내 `style` 사용)
| 속성 | 설명 |
|------|------|
| `font-size` | 글자 크기 지정 (예: `14px`, `18px`) |
| `color` | 글자 색상 지정 (예: `red`, `#0044ff`) |
| `background-color` | 배경색 및 투명도 조절 (예: `rgba(255,255,255,0.6)`) |
| `border-radius` | 텍스트 상자 모서리를 둥글게 만듦 |
| `width / height` | 텍스트 상자의 크기 조정 |
| `text-align` | 텍스트 정렬 (`left`, `center`, `right`) |

> 💡 `DivIcon`을 이용하면 지도 위에 지명, 수치값, 태그 등  
> **데이터 기반 텍스트 라벨링(labeling)** 을 시각적으로 표현할 수 있습니다.

---

In [ ]:
# 지도 위에 HTML 라벨 표시하기 (DivIcon)
import folium

# 경복궁 좌표
loc = [37.5796, 126.9770]

# 지도 생성
map3 = folium.Map(location=loc, zoom_start=16)

# 텍스트용 HTML 블록 정의
html_label = '''
<div style="
    font-size:16px;
    color:#0044ff;
    background-color:rgba(255,255,255,0.8);
    border-radius:5px;
    text-align:center;
    width:120px;
    padding:3px;">
<b>경복궁</b>
</div>
'''

# DivIcon 사용하여 지도에 추가
folium.Marker(
    location=loc,
    icon=folium.DivIcon(html=html_label)
).add_to(map3)

print("HTML 스타일 라벨 추가 완료!")
map3

HTML 스타일 라벨 추가 완료!


---

## 4️⃣ 반복문(for)을 이용한 다중 마커 표시

`for` 반복문과 DataFrame을 조합하면  
수십~수백 개의 지점을 **자동으로 지도에 표시**할 수 있습니다.

| 개념 | 설명 |
|------|------|
| `df.iterrows()` | DataFrame의 각 행(row)을 반복 처리할 때 사용 |
| `iloc` / `loc` | 인덱스 또는 라벨 기반으로 개별 좌표를 참조 |
| `add_to(map)` | 만들어진 마커를 현재 지도 객체에 추가 |

> 💡 `for`문 내에서 조건문(`if`)을 함께 사용하면,  
> 예를 들어 특정 조건(`관객 수 > 100만`)에만 다른 색상의 마커를 표시하는 것도 가능합니다.

---

In [ ]:
# 🔁 반복문을 이용한 여러 지점 마커 표시
import folium
import pandas as pd

# 제주 관광지 예시 데이터
data = {
    "이름": ["용두암", "성산일출봉", "정방폭포", "중문관광단지", "한라산1100고지", "차귀도"],
    "위도": [33.5163, 33.4583, 33.2458, 33.2554, 33.3595, 33.3040],
    "경도": [126.5110, 126.9424, 126.5623, 126.4122, 126.5129, 126.1621]
}
df = pd.DataFrame(data)
print("관광지 데이터프레임 미리보기:")
print(df.head(), "\n")

# 지도 중심 계산
center = [df["위도"].mean(), df["경도"].mean()]
print("지도 중심:", center)

# 지도 객체 생성
map4 = folium.Map(location=center, zoom_start=10)

# 반복문을 이용해 마커 추가
for i, row in df.iterrows():
    loc = [row["위도"], row["경도"]]
    name = row["이름"]

    # 마커 색상 변경 규칙 (짝수행/홀수행)
    color = "red" if i % 2 == 0 else "blue"

    folium.Marker(
        location=loc,
        popup=f"{name} (색상: {color})",
        icon=folium.Icon(color=color, icon="star")
    ).add_to(map4)

print("모든 관광지 마커 추가 완료!")
map4

관광지 데이터프레임 미리보기:
          이름       위도        경도
0        용두암  33.5163  126.5110
1      성산일출봉  33.4583  126.9424
2       정방폭포  33.2458  126.5623
3     중문관광단지  33.2554  126.4122
4  한라산1100고지  33.3595  126.5129 

지도 중심: [np.float64(33.356550000000006), np.float64(126.51715)]
모든 관광지 마커 추가 완료!


---

## 5️⃣ CircleMarker로 데이터의 “크기” 표현

`CircleMarker`는 원의 반지름(radius)을 수치 데이터에 비례하도록 설정할 수 있어,  
**값의 크기 시각화(예: 인구수, 매출액, 방문자 수)** 에 매우 유용합니다.

| 속성명 | 설명 |
|--------|------|
| `radius` | 원의 반지름 (데이터 값에 비례시킬 수 있음) |
| `color` | 원의 테두리 색상 |
| `fill` | 내부 색상 채움 여부 (기본값: `True`) |
| `fill_color` | 내부 색상 지정 |
| `fill_opacity` | 내부 색상의 투명도 (0~1 사이 값) |
| `stroke` | 테두리선 표시 여부 |

> 💡 **데이터 크기를 시각화할 때**  
> 단순히 수치를 직접 반지름으로 쓰면 원이 너무 커지므로,  
> 보통 `radius = 값 ** 0.5` 또는 `값 / 일정 상수` 형태로 스케일링합니다.

---

In [ ]:
# 🧮 CircleMarker로 값의 크기를 반영한 지도 만들기
import folium
import pandas as pd

# 풍속 데이터 예시 (간단한 6개 지점)
df = pd.DataFrame({
    "도시": ["서울", "부산", "대구", "광주", "인천", "춘천"],
    "위도": [37.57, 35.17, 35.87, 35.16, 37.45, 37.87],
    "경도": [126.98, 129.07, 128.60, 126.85, 126.70, 127.73],
    "풍속": [10.5, 8.2, 12.8, 9.0, 6.3, 15.0]
})
print("풍속 데이터:")
print(df, "\n")

# 지도 중심 계산
center = [df["위도"].mean(), df["경도"].mean()]
print("지도 중심:", center)

# 지도 생성
map5 = folium.Map(location=center, zoom_start=6)

# 각 도시의 풍속에 비례한 원형 표시
for i, row in df.iterrows():
    loc = [row["위도"], row["경도"]]
    radius = row["풍속"] ** 0.8  # 값의 크기를 조정하여 원 크기 조절
    folium.CircleMarker(
        location=loc,
        radius=radius,          # 원의 반지름
        color="red",            # 테두리 색
        fill=True,              # 내부 색 채움
        fill_color="red",
        fill_opacity=0.5,       # 투명도
        popup=f"{row['도시']} 풍속: {row['풍속']} m/s"
    ).add_to(map5)
    print(f"{row['도시']} 마커 추가 (풍속={row['풍속']})")

print("모든 CircleMarker 추가 완료!")
map5

풍속 데이터:
   도시     위도      경도    풍속
0  서울  37.57  126.98  10.5
1  부산  35.17  129.07   8.2
2  대구  35.87  128.60  12.8
3  광주  35.16  126.85   9.0
4  인천  37.45  126.70   6.3
5  춘천  37.87  127.73  15.0 

지도 중심: [np.float64(36.51500000000001), np.float64(127.65500000000002)]
서울 마커 추가 (풍속=10.5)
부산 마커 추가 (풍속=8.2)
대구 마커 추가 (풍속=12.8)
광주 마커 추가 (풍속=9.0)
인천 마커 추가 (풍속=6.3)
춘천 마커 추가 (풍속=15.0)
모든 CircleMarker 추가 완료!


---

## 6️⃣ 시각적 통일성과 해석력 높이기

1. **지도 색상 팔레트 유지**  
   - 동일한 범주 데이터(예: 도시, 지역)는 같은 색상을 유지합니다.  
2. **글자 크기(font-size) 통일**  
   - 지도 내 모든 라벨은 12~14px 수준으로 통일하면 가독성이 좋아집니다.  
3. **투명도 조절로 중복 방지**  
   - 마커가 많을 경우 `fill_opacity=0.5`처럼 반투명하게 설정하면 시각적으로 깔끔합니다.  
4. **범례(Legend)**  
   - Folium은 기본적으로 범례 기능이 없지만, HTML을 삽입하여 직접 만들 수 있습니다.  
5. **확대/축소 제어 버튼 추가**  
   - `zoom_control=True`로 설정 시 사용자 조작이 가능합니다.
---

In [ ]:
# 🌈 Folium DivIcon 스타일링 심화
import folium

# 중심 좌표
loc = [37.57, 126.98]
map6 = folium.Map(location=loc, zoom_start=14)

# 다양한 스타일을 가진 라벨 생성
html_label = '''
<div style="
    font-size:20px;
    font-weight:bold;
    color:#ffffff;
    background-color:rgba(0, 100, 255, 0.7);
    border: 2px solid #ffffff;
    border-radius: 10px;
    padding: 6px;
    text-align:center;
    width:140px;">
데이터사이언스관
</div>
'''

folium.Marker(
    location=loc,
    icon=folium.DivIcon(html=html_label)
).add_to(map6)

print("고급 스타일 라벨 추가 완료! (배경색, 테두리, 글씨 크기 포함)")
map6

고급 스타일 라벨 추가 완료! (배경색, 테두리, 글씨 크기 포함)


# 🎯 퀴즈: 덕성여자대학교 지도 표시하기 (기초)

지금까지 배운 Folium 기본기를 활용해 아주 간단한 문제를 풀어보세요! 🌸

---

## 🧭 문제
아래 조건에 따라 **덕성여자대학교**를 지도 위에 표시하세요.

1. 중심 좌표는 덕성여자대학교로 설정합니다.  
   - 위도(`lat`) = 37.6525  
   - 경도(`lon`) = 127.0165  
2. `zoom_start`는 **16**으로 지정합니다.  
3. 마커(`folium.Marker`)를 이용해 지도 위에  
   `덕성여자대학교` 라는 문구를 표시하세요.  
4. 마지막 줄에 “지도 완성!”을 출력하세요.

---

## 💡 힌트
- `folium.Map(location=[위도, 경도], zoom_start=16)`  
- `folium.Marker(location=[위도, 경도], popup="덕성여자대학교").add_to(map)`  
- `map`을 실행하면 지도가 표시됩니다.

---

🧠 **보너스 도전**
- 마커 색상을 `pink`로 바꿔보세요.  
- 클릭 시 영어 이름 `"Duksung Women's University"` 가 보이도록 설정해보세요.

---

🎯 출력 예시  
- 지도 위에 덕성여자대학교 마커가 나타나면 성공입니다!  
- 하단에 “지도 완성!”이 출력되어야 합니다.

In [ ]:
map = folium.Map(location=[37.6525, 127.0165], zoom_start=16)

In [ ]:
# 1) 지도 중심 좌표 설정
center = [37.6525, 127.0165]  # 서울 종로구 근처
print("지도 중심 좌표:", center)

지도 중심 좌표: [37.6525, 127.0165]


In [ ]:
# 2) 지도 객체 생성
# - location: 중심 좌표
# - zoom_start: 확대 레벨 (1=전세계, 13=도시 수준, 16=건물 수준)
map = folium.Map(location=center, zoom_start=13)

In [ ]:
map

In [1]:
# 지도 위에 HTML 라벨 표시하기 (DivIcon)
import folium

# 경복궁 좌표
loc = [37.6525, 127.0165]

# 지도 생성
map3 = folium.Map(location=loc, zoom_start=16)

# 텍스트용 HTML 블록 정의
html_label = '''
<div style="
    font-size:16px;
    color:#0044ff;
    background-color:rgba(255,255,255,0.8);
    border-radius:5px;
    text-align:center;
    width:120px;
    padding:3px;">
<b>Duksung women's University</b>
</div>
'''

# DivIcon 사용하여 지도에 추가
folium.Marker(
    location=loc,
    icon=folium.DivIcon(html=html_label)
).add_to(map3)

folium.Marker(
    location=loc,
    popup="이 마커는 드래그 가능!",
    draggable=True,
    icon=folium.Icon(color="pink", icon="flag")
).add_to(map3)

print("지도 완성!")
map3

지도 완성!
